# BLIŽINA, EKSCENTRIČNOST IN VMESNA CENTRALNOST

Pri najinem projektu bova s pomočjo treh različnih mer (ekscentričnosti, bližine in vmesne centralnosti) iskala in analizirala najpomembnejša vozlišča v grafih oziroma socialnih omrežjih. Vozlišče je pomembnejše, če ima višjo vrednost bližine in vmesne centralnosti ter nižjo vrednost ekscentričnosti. Natančnejši opisi pojmov so spodaj. Primerjala bova, kako pogosto je vozlišče, ki je pomembno z vidika ene mere, pomembno tudi v okviru ostalih dveh mer.

Najino glavno orodje za analizo grafov bo $Sage$, kjer bova generirala več grafov (približno 1000) v različnih velikostih (približno 10), grafe socialnih omrežij pa bova pridobila s spleta. Opazovala bova tudi, kako se vrednosti mer za najpomembnejša vozlišča spremenijo, če se omejimo na podgraf v določenem grafu. Grafi, ki jih bova pri projektu analizirala, bodo neusmerjeni.

## Bližina
Bližina je v povezanem grafu mera centralnosti, ki jo izračunamo kot
recipročno vsoto dolžin najkrajših poti med nekim vozliščem in vsemi
drugimi vozlišči v grafu. Bližje kot je opazovano vozlišče ostalim
vozliščem v grafu, bolj centralno je.
C(x) = \frac{1}{\sum_{y}^{} d(y,x)},
kjer je *d*(*y*, *x*) razdalja med vozliščema x in y. Pogosto se namesto
zgornje vrednosti izračuna povprečno dolžino najkrajše poti v grafu.
Dobimo jo tako, da zgornjo formulo pomnožimo z *N* − 1, kjer je *N*
število vseh vozlišč v grafu. Pri obsežnejših grafih se  − 1 izpusti iz
enačbe, zato se za bližino uporablja kar sledečo formulo:
$$C(x) = \\frac{N}{\\sum\_{y}^{} d(y,x)}.$$

Pri usmerjenih grafih je potrebno upoštevati tudi smer povezav. Določeno
vozlišče ima lahko različno bližino za vhodne in izhodne povezave. V
nepovezanih grafih namesto recipročne vsote dolžin najkrajših poti med
vozlišči računamo vsoto recipročnih dolžin najkrajših poti med vozlišči.
Pri tem upoštevamo, da $\\frac{1}{\\infty} = 0.$
$$H(x) = \\sum\_{y \\ne x}^{} \\frac{1}{d(y, x)}.$$

## Implementacija algoritma
Torej najprej se bova osredotočila na implementacijo algoritma, ki bo osnovan na podlagi Dijkstrovega algoritma, ki mu bova dodala vse tri mere, glede na katere bova ocenjevala grafe.

In [13]:
from collections import deque, namedtuple


# uporabimo neskončno za začetno razdaljo do vozlišč.
inf = float('inf')
#trojka z začetnm vozliščem, končnim vozliščem in ceno povezave
Edge = namedtuple('Edge', 'start, end, cost')


def make_edge(start, end, cost=1):
  return Edge(start, end, cost)


class Graph:
    def __init__(self, edges):
        # preverimo če so pravi podatki torej če lahko iz tega zgradimo graf
        wrong_edges = [i for i in edges if len(i) not in [2, 3]]
        if wrong_edges:
            raise ValueError('Wrong edges data: {}'.format(wrong_edges))

        self.edges = [make_edge(*edge) for edge in edges]

    @property
    def vertices(self):
        #dobimo množico vozlišč, ki so v grafu
        return set(
            sum(
                ([edge.start, edge.end] for edge in self.edges), []
            )
        )

    def get_node_pairs(self, n1, n2, both_ends=True):
        #dobimo pare vozlišč
        if both_ends:
            node_pairs = [[n1, n2], [n2, n1]]
        else:
            node_pairs = [[n1, n2]]
        return node_pairs

    def remove_edge(self, n1, n2, both_ends=True):
        #funkcija, ki odstrani povezavo med vozliščema
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        edges = self.edges[:]
        for edge in edges:
            if [edge.start, edge.end] in node_pairs:
                self.edges.remove(edge)

    def add_edge(self, n1, n2, cost=1, both_ends=True):
        #funkcija, ki doda povezavo med vozliščema
        node_pairs = self.get_node_pairs(n1, n2, both_ends)
        for edge in self.edges:
            if [edge.start, edge.end] in node_pairs:
                return ValueError('Edge {} {} already exists'.format(n1, n2))

        self.edges.append(Edge(start=n1, end=n2, cost=cost))
        if both_ends:
            self.edges.append(Edge(start=n2, end=n1, cost=cost))

    @property
    def neighbours(self):
        #funkcija, ki najde sosede vsakega vozlišča
        neighbours = {vertex: set() for vertex in self.vertices}
        for edge in self.edges:
            neighbours[edge.start].add((edge.end, edge.cost))

        return neighbours

    def dijkstra(self, source, dest):
        assert source in self.vertices, 'Such source node doesn\'t exist'
        #nastavimo vse razdalje na začetku na neskončno
        distances = {vertex: inf for vertex in self.vertices}
        #prejšna vozlišča, nazačetki vse na None
        previous_vertices = {
            vertex: None for vertex in self.vertices
        }
        #razdalja od začetka do začetka je 0
        distances[source] = 0
        #naredimo kopijo
        vertices = self.vertices.copy()
        #dokler imamo za preverit še kakšno vozlišle
        while vertices:
            #trenutno vozlišče
            current_vertex = min(
                vertices, key=lambda vertex: distances[vertex])
            vertices.remove(current_vertex)
            if distances[current_vertex] == inf:
                break
            for neighbour, cost in self.neighbours[current_vertex]:
                alternative_route = distances[current_vertex] + cost
                if alternative_route < distances[neighbour]:
                    distances[neighbour] = alternative_route
                    previous_vertices[neighbour] = current_vertex

        path, current_vertex = deque(), dest
        while previous_vertices[current_vertex] is not None:
            path.appendleft(current_vertex)
            current_vertex = previous_vertices[current_vertex]
        if path:
            path.appendleft(current_vertex)
        return path


graph = Graph([
    ("a", "b"),("a", "f"), ("b", "c"),
    ("b", "d"), ("c", "d"), ("c", "f"),  ("d", "e"),
    ("e", "f")])

graph.dijkstra("a", "e")

deque(['a', 'b', 'd', 'e'])